In [2]:
!pip install transformers datasets torch torchvision
!pip install 'accelerate>={ACCELERATE_MIN_VERSION}'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
ERROR: Invalid requirement: 'accelerate>={ACCELERATE_MIN_VERSION}': Expected end or semicolon (after name and no valid version specifier)
    accelerate>={ACCELERA

## Task 2 ##

In [3]:

from datasets import load_dataset

dataset = load_dataset("sem_eval_2018_task_1", "subtask5.english")

print(dataset["validation"][-1])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sem_eval_2018_task_1.py:   0%|          | 0.00/6.29k [00:00<?, ?B/s]

The repository for sem_eval_2018_task_1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sem_eval_2018_task_1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/6838 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3259 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/886 [00:00<?, ? examples/s]

{'ID': '2018-En-03386', 'Tweet': 'I am really flattered and happy to hear those complements for my blog! You guys motivates me to write more for my blog. Thank you! sml 💞', 'anger': False, 'anticipation': False, 'disgust': False, 'fear': False, 'joy': True, 'love': False, 'optimism': True, 'pessimism': False, 'sadness': False, 'surprise': False, 'trust': False}


## Task 3 ##

In [4]:
from transformers import BertTokenizer
import torch

def tokenize_data(dataset):
    # Load the BERT tokenizer
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    # Define the tokenization function
    def preprocess(example):
        # Tokenize the text
        encoding = tokenizer(
            example["Tweet"],
            padding="max_length",  # Pad to max length
            truncation=True,       # Truncate if too long
            max_length=128,        # Define max token length
        )
        # Convert labels to a tensor of floats
        labels = [example[label] for label in dataset["train"].features.keys() if label not in ["ID", "Tweet"]]
        encoding["labels"] = torch.tensor(labels, dtype=torch.float)
        return encoding

    # Apply the tokenization function to the dataset
    encoded_dataset = dataset.map(preprocess, batched=False)

    # Set format to PyTorch
    encoded_dataset.set_format("torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])

    return encoded_dataset


# Tokenize the dataset
encoded_dataset = tokenize_data(dataset)

# Print the keys of the last data point in the validation set
print(encoded_dataset["validation"][-1].keys())


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/6838 [00:00<?, ? examples/s]

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

Map:   0%|          | 0/886 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


## Task 4 ##

In [6]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import torch

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the number of labels
num_labels = len(dataset["train"].features) - 2  # Exclude "ID" and "Tweet"

# Create id2label and label2id mappings
id2label = {i: label for i, label in enumerate([key for key in dataset["train"].features.keys() if key not in ["ID", "Tweet"]])}
label2id = {label: i for i, label in id2label.items()}

# Define the model and move it to the GPU (if available)
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification"
).to(device)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions
    preds = (preds > 0.5).astype(int)  # Convert logits to binary predictions

    # Calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="micro")
    accuracy = accuracy_score(labels, preds)

    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    # Enable mixed precision if possible (this also speeds up training)
    fp16=True if torch.cuda.is_available() else False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.332500,0.316661,0.248307,0.647108,0.816643,0.535863
2,0.259300,0.302272,0.257336,0.662230,0.826298,0.552522
3,0.260200,0.301693,0.278781,0.680446,0.825198,0.578899


TrainOutput(global_step=2565, training_loss=0.29860036517212035, metrics={'train_runtime': 368.8189, 'train_samples_per_second': 55.621, 'train_steps_per_second': 6.955, 'total_flos': 1349474086052352.0, 'train_loss': 0.29860036517212035, 'epoch': 3.0})

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Extract tweets and labels
tweets = dataset["train"]["Tweet"]
labels = np.array([list(example.values())[2:] for example in dataset["train"]])  # Exclude "ID" and "Tweet"

# Split into train and validation sets
tweets_train, tweets_val, labels_train, labels_val = train_test_split(tweets, labels, test_size=0.2, random_state=42)

# TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(tweets_train)
X_val = vectorizer.transform(tweets_val)

# Logistic Regression with OneVsRest for multi-label classification
clf = OneVsRestClassifier(LogisticRegression(max_iter=1000))
clf.fit(X_train, labels_train)

# Evaluate
predictions = clf.predict(X_val)
print(classification_report(labels_val, predictions, target_names=id2label.values()))


              precision    recall  f1-score   support

       anger       0.83      0.53      0.64       507
anticipation       0.50      0.01      0.02       200
     disgust       0.78      0.47      0.59       516
        fear       0.93      0.26      0.41       283
         joy       0.87      0.55      0.67       507
        love       0.82      0.10      0.18       136
    optimism       0.76      0.38      0.51       400
   pessimism       0.33      0.01      0.01       166
     sadness       0.82      0.27      0.41       424
    surprise       1.00      0.03      0.05        76
       trust       0.00      0.00      0.00        71

   micro avg       0.82      0.35      0.49      3286
   macro avg       0.69      0.24      0.32      3286
weighted avg       0.77      0.35      0.46      3286
 samples avg       0.55      0.37      0.42      3286



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_